# 注意力機制與Transformer模型

循環神經網絡的一個主要缺點是序列中的所有詞語對結果的影響相同。這導致標準LSTM編碼器-解碼器模型在序列到序列任務（例如命名實體識別和機器翻譯）中的表現不佳。實際上，輸入序列中的某些特定詞語通常比其他詞語對序列輸出有更大的影響。

考慮一個序列到序列模型，例如機器翻譯。它由兩個循環神經網絡實現，其中一個網絡（**編碼器**）將輸入序列壓縮到隱藏狀態中，另一個網絡（**解碼器**）將隱藏狀態展開為翻譯結果。這種方法的問題在於，網絡的最終狀態很難記住句子的開頭部分，因此在處理長句子時模型的質量會下降。

**注意力機制**提供了一種方法，能夠對每個輸入向量在RNN的每個輸出預測中的上下文影響進行加權。其實現方式是通過在輸入RNN的中間狀態和輸出RNN之間創建捷徑。在生成輸出符號$y_t$時，我們會考慮所有輸入隱藏狀態$h_i$，並使用不同的權重係數$\alpha_{t,i}$。

![顯示具有加性注意力層的編碼器/解碼器模型的圖片](../../../../../translated_images/zh-MO/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau等人，2015](https://arxiv.org/pdf/1409.0473.pdf)中的加性注意力機制編碼器-解碼器模型，圖片引用自[這篇博客文章](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

注意力矩陣$\{\alpha_{i,j}\}$表示某些輸入詞語在生成輸出序列中的某個詞語時所起的作用程度。以下是這樣一個矩陣的示例：

![顯示RNNsearch-50找到的示例對齊的圖片，取自Bahdanau - arviz.org](../../../../../translated_images/zh-MO/bahdanau-fig3.09ba2d37f202a6af.webp)

*圖片取自[Bahdanau等人，2015](https://arxiv.org/pdf/1409.0473.pdf)（圖3）*

注意力機制是目前或接近目前自然語言處理領域的最先進技術的核心。添加注意力機制會大幅增加模型參數的數量，這導致了RNN的擴展問題。RNN擴展的一個關鍵限制是模型的循環特性使得批量處理和訓練並行化變得困難。在RNN中，序列的每個元素都需要按順序處理，這意味著它無法輕易並行化。

注意力機制的採用結合了這一限制，促成了如今我們所熟知和使用的最先進Transformer模型的誕生，從BERT到OpenGPT3。

## Transformer模型

與將每次預測的上下文傳遞到下一個評估步驟不同，**Transformer模型**使用**位置編碼**和**注意力**來捕捉給定輸入在提供的文本窗口中的上下文。下圖展示了位置編碼與注意力如何在給定窗口中捕捉上下文。

![顯示Transformer模型中如何進行評估的動畫GIF](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

由於每個輸入位置可以獨立映射到每個輸出位置，Transformer模型比RNN更容易並行化，這使得能夠構建更大、更具表達力的語言模型。每個注意力頭可以用來學習詞語之間的不同關係，從而改善下游的自然語言處理任務。

## 構建簡單的Transformer模型

Keras中並不包含內建的Transformer層，但我們可以自己構建。與之前一樣，我們將專注於AG News數據集的文本分類，但值得一提的是，Transformer模型在更困難的自然語言處理任務中表現最佳。


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

新的層在 Keras 中應該繼承 `Layer` 類別，並實現 `call` 方法。我們先從 **位置嵌入** 層開始。我們將使用[官方 Keras 文件中的一些代碼](https://keras.io/examples/nlp/text_classification_with_transformer/)。我們假設所有輸入序列都填充到長度 `maxlen`。


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

這一層包含兩個 `Embedding` 層：一個用於嵌入標記（如我們之前討論的方式），另一個用於嵌入標記的位置。標記位置是通過使用 `tf.range` 創建從 0 到 `maxlen` 的自然數序列，然後將其傳遞到嵌入層中。接著，將兩個生成的嵌入向量相加，產生形狀為 `maxlen`$\times$`embed_dim` 的位置嵌入表示。

現在，我們來實現 transformer 區塊。它將接收之前定義的嵌入層的輸出：


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

現在，我們準備好定義完整的 Transformer 模型：


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer 模型

**BERT**（雙向編碼器表示來自 Transformer）是一個非常大型的多層 Transformer 網絡，*BERT-base* 有 12 層，*BERT-large* 則有 24 層。該模型首先在大量文本數據（維基百科 + 書籍）上進行無監督訓練（預測句子中的被遮蔽詞）。在預訓練過程中，模型吸收了大量的語言理解能力，之後可以通過微調與其他數據集結合使用。這個過程被稱為 **遷移學習**。

![圖片來源：http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/zh-MO/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Transformer 架構有許多變體，包括 BERT、DistilBERT、BigBird、OpenGPT3 等，它們都可以進行微調。

接下來，我們來看看如何使用預訓練的 BERT 模型來解決我們傳統的序列分類問題。我們將借用一些想法和代碼，來自[官方文檔](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)。

為了加載預訓練模型，我們將使用 **Tensorflow hub**。首先，讓我們加載 BERT 專用的向量化工具：


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

使用與原始網絡訓練時相同的向量化工具是非常重要的。此外，BERT 向量化工具會返回三個組件：
* `input_word_ids`，這是一個表示輸入句子中標記編號的序列
* `input_mask`，用於顯示序列中哪些部分包含實際輸入，哪些部分是填充。它類似於由 `Masking` 層生成的遮罩
* `input_type_ids` 用於語言建模任務，允許在一個序列中指定兩個輸入句子。

接著，我們可以實例化 BERT 特徵提取器：


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


所以，BERT 層會返回一些有用的結果：
* `pooled_output` 是通過平均序列中所有 token 的結果。你可以將其視為整個網絡的智能語義嵌入。它等同於我們之前模型中的 `GlobalAveragePooling1D` 層的輸出。
* `sequence_output` 是最後一個 transformer 層的輸出（對應於我們上面模型中的 `TransformerBlock` 的輸出）。
* `encoder_outputs` 是所有 transformer 層的輸出。由於我們載入的是 4 層的 BERT 模型（從名稱中包含 `4_H` 可以推測出來），它有 4 個張量。最後一個張量與 `sequence_output` 相同。

現在我們將定義端到端的分類模型。我們將使用*函數式模型定義*，在定義模型輸入後，提供一系列表達式來計算其輸出。我們還會將 BERT 模型的權重設置為不可訓練，只訓練最終的分類器：


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


儘管可訓練的參數很少，過程仍然相當緩慢，因為 BERT 特徵提取器的計算量非常大。看起來我們無法達到合理的準確度，可能是因為訓練不足，或者模型參數不足。

讓我們嘗試解凍 BERT 的權重並進行訓練。這需要非常小的學習率，並且需要更謹慎的訓練策略，包括使用 **warmup** 和 **AdamW** 優化器。我們將使用 `tf-models-official` 套件來創建優化器：


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

如你所見，訓練過程相當緩慢——但你可能想嘗試進行幾個世代（5-10）的訓練，並比較我們之前使用的方法，看看是否能獲得最佳結果。

## Huggingface Transformers 庫

另一種非常常見（且稍微簡單）的使用 Transformer 模型的方法是 [HuggingFace 套件](https://github.com/huggingface/)，它為不同的 NLP 任務提供了簡單的構建模塊。此套件同時支持 Tensorflow 和 PyTorch，後者是另一個非常流行的神經網絡框架。

> **注意**：如果你對了解 Transformers 庫的運作方式不感興趣——你可以跳到筆記本的最後部分，因為你不會看到任何與我們之前所做的有實質性不同的內容。我們將重複使用不同的庫和更大的模型來訓練 BERT 模型的相同步驟。因此，這個過程涉及一些相當長時間的訓練，所以你可能只想瀏覽一下代碼。

讓我們看看如何使用 [Huggingface Transformers](http://huggingface.co) 解決我們的問題。


首先，我們需要選擇要使用的模型。除了內建的模型之外，Huggingface 還有一個[線上模型庫](https://huggingface.co/models)，社群提供了許多預訓練模型。在這些模型中，只需提供模型名稱即可載入並使用。所有模型所需的二進位檔案都會自動下載。

有時候，你可能需要載入自己的模型。在這種情況下，你可以指定包含所有相關檔案的目錄，包括 tokenizer 的參數、`config.json` 文件中的模型參數、二進位權重等。

透過模型名稱，我們可以實例化模型和 tokenizer。讓我們先從 tokenizer 開始：


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` 對象包含可直接用於編碼文本的 `encode` 函數：


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

我們也可以使用分詞器將序列編碼為適合傳遞給模型的方式，即包括 `token_ids`、`input_mask` 欄位等。我們還可以通過提供 `return_tensors='tf'` 參數來指定我們想要 Tensorflow 張量：


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

在我們的案例中，我們將使用名為 `bert-base-uncased` 的預訓練 BERT 模型。*Uncased* 表示該模型對大小寫不敏感。

在訓練模型時，我們需要提供已分詞的序列作為輸入，因此我們將設計數據處理管道。由於 `tokenizer.encode` 是一個 Python 函數，我們將採用與上一單元相同的方法，使用 `py_function` 來調用它：


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

現在我們可以使用 `BertForSequenceClassfication` 套件載入實際模型。這確保我們的模型已經具備分類所需的架構，包括最終的分類器。您會看到警告訊息，指出最終分類器的權重尚未初始化，且模型需要進行預訓練——這完全沒問題，因為這正是我們即將進行的操作！


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


從 `summary()` 中可以看到，該模型包含了將近 1.1 億個參數！假設我們想在相對較小的數據集上進行簡單的分類任務，我們可能不希望訓練 BERT 基層：


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


現在我們準備開始訓練了！

> **注意**：訓練完整規模的 BERT 模型可能會非常耗時！因此，我們只會訓練前 32 個批次。這只是為了展示模型訓練的設置方式。如果你有興趣嘗試完整規模的訓練，只需移除 `steps_per_epoch` 和 `validation_steps` 參數，然後準備耐心等待吧！


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


如果你增加迭代次數並耐心等待，並進行多個訓練週期，你可以期待 BERT 分類能夠提供最佳的準確率！這是因為 BERT 已經相當了解語言的結構，我們只需要微調最終的分類器。然而，由於 BERT 是一個大型模型，整個訓練過程需要很長的時間，並且需要強大的計算能力！（GPU，最好是多個）。

> **Note:** 在我們的範例中，我們使用的是最小的預訓練 BERT 模型之一。還有更大的模型可能會產生更好的結果。


## 重點

在本單元中，我們探討了基於**transformers**的最新模型架構。我們將其應用於文本分類任務，但同樣地，BERT模型也可以用於實體抽取、問答系統以及其他自然語言處理任務。

Transformer模型代表了自然語言處理領域的最新技術，在大多數情況下，當您開始實現自定義自然語言處理解決方案時，這應該是您首先嘗試的解決方案。然而，如果您希望構建更高級的神經網絡模型，理解本模組中討論的循環神經網絡的基本原理是非常重要的。



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
